In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import openai
import os

load_dotenv(dotenv_path="config.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()
def message_to_openai(message):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        store=True,
        messages=[
            {"role":"system", "content":"한국어로 5문장으로 대답해주세요."},
            {"role": "user", "content": message}
        ]
    )
    return completion

In [28]:
ai_terms = [
    "인공지능 (Artificial Intelligence, AI)",
    "머신러닝 (Machine Learning, ML)",
    "딥러닝 (Deep Learning)",
    "데이터 마이닝 (Data Mining)",
    "지도학습 (Supervised Learning)",
    "비지도학습 (Unsupervised Learning)",
    "강화학습 (Reinforcement Learning, RL)",
    "전이학습 (Transfer Learning)",
    "자기지도학습 (Self-Supervised Learning)",
    "생성 모델 (Generative Model)",
    "확률 분포 (Probability Distribution)",
    "베이즈 정리 (Bayes' Theorem)",
    "선형대수 (Linear Algebra)",
    "행렬 연산 (Matrix Operations)",
    "미분 (Differentiation)",
    "편미분 (Partial Derivative)",
    "경사 하강법 (Gradient Descent)",
    "손실 함수 (Loss Function)",
    "비용 함수 (Cost Function)",
    "최적화 (Optimization)",
    "뉴런 (Neuron)",
    "퍼셉트론 (Perceptron)",
    "다층 퍼셉트론 (Multilayer Perceptron, MLP)",
    "합성곱 신경망 (Convolutional Neural Network, CNN)",
    "순환 신경망 (Recurrent Neural Network, RNN)",
    "장단기 기억 네트워크 (Long Short-Term Memory, LSTM)",
    "게이트 순환 유닛 (Gated Recurrent Unit, GRU)",
    "트랜스포머 (Transformer)",
    "인코더-디코더 모델 (Encoder-Decoder Model)",
    "BERT (Bidirectional Encoder Representations from Transformers)",
    "활성화 함수 (Activation Function)",
    "ReLU (Rectified Linear Unit)",
    "시그모이드 (Sigmoid)",
    "소프트맥스 (Softmax)",
    "배치 정규화 (Batch Normalization)",
    "드롭아웃 (Dropout)",
    "학습률 (Learning Rate)",
    "과적합 (Overfitting)",
    "데이터 증강 (Data Augmentation)",
    "모델 정규화 (Model Regularization)",
    "토큰화 (Tokenization)",
    "임베딩 (Embedding)",
    "워드투벡 (Word2Vec)",
    "글로브 (GloVe)",
    "문장피스 (SentencePiece)",
    "BPE (Byte Pair Encoding)",
    "언어 모델 (Language Model)",
    "감성 분석 (Sentiment Analysis)",
    "개체명 인식 (Named Entity Recognition, NER)",
    "기계 번역 (Machine Translation)",
    "이미지 분류 (Image Classification)",
    "객체 탐지 (Object Detection)",
    "세그멘테이션 (Segmentation)",
    "합성곱 (Convolution)",
    "풀링 (Pooling)",
    "YOLO (You Only Look Once)",
    "ResNet (Residual Network)",
    "VGGNet (Visual Geometry Group Network)",
    "GAN (Generative Adversarial Network)",
    "Autoencoder (자동 인코더)",
    "확률적 경사 하강법 (Stochastic Gradient Descent, SGD)",
    "Adam 옵티마이저 (Adam Optimizer)",
    "RMSprop",
    "모멘텀 (Momentum)",
    "하이퍼파라미터 튜닝 (Hyperparameter Tuning)",
    "초기화 방법 (Weight Initialization)",
    "전역 최적화 (Global Optimization)",
    "군집 분석 (Clustering)",
    "앙상블 학습 (Ensemble Learning)",
    "랜덤 포레스트 (Random Forest)",
    "정확도 (Accuracy)",
    "정밀도 (Precision)",
    "재현율 (Recall)",
    "F1 점수 (F1 Score)",
    "AUC-ROC 곡선 (AUC-ROC Curve)",
    "평균 제곱 오차 (Mean Squared Error, MSE)",
    "교차 엔트로피 손실 (Cross-Entropy Loss)",
    "BLEU 점수 (BLEU Score)",
    "ROUGE 점수 (ROUGE Score)",
    "혼동 행렬 (Confusion Matrix)",
    "대규모 언어 모델 (Large Language Model, LLM)",
    "ChatGPT",
    "인공지능 윤리 (AI Ethics)",
    "연합 학습 (Federated Learning)",
    "지속 학습 (Continual Learning)",
    "파운데이션 모델 (Foundation Model)",
    "미세 조정 (Fine-Tuning)",
    "제로샷 학습 (Zero-Shot Learning)",
    "소수샷 학습 (Few-Shot Learning)",
    "체인 오브 띠로우트 (Chain-of-Thought)",
    "클라우드 AI (Cloud AI)",
    "API 기반 AI 서비스 (AI-as-a-Service)",
    "데이터 레이블링 (Data Labeling)",
    "MLOps (Machine Learning Operations)",
    "Explainable AI (XAI, 설명 가능한 AI)",
    "시계열 분석 (Time Series Analysis)",
    "오픈소스 프레임워크 (TensorFlow, PyTorch)",
    "ONNX (Open Neural Network Exchange)",
    "AutoML (자동화 머신러닝)",
    "AI 모델 배포 (AI Model Deployment)"
]


In [20]:
from tqdm import tqdm
import json

prompt_template = "{term}이 무엇인지 설명해줘"
result =[]
for item in tqdm(ai_terms):
    ai_terms_dict = {"instruction": None, "input":"", "output":None}
    prompt = prompt_template.format(term=item)
    response = message_to_openai(prompt)
    ai_terms_dict['instruction'] = prompt
    ai_terms_dict["output"] = response.choices[0].message.content
    result.append(ai_terms_dict)

# JSON 파일로 저장
with open("corpus.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

100%|██████████| 100/100 [07:21<00:00,  4.41s/it]


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# JSON 데이터 로드
dataset = load_dataset("json", data_files="corpus.json")["train"]

# 데이터 분할 (70% Train, 20% Validation, 10% Test)
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, train_size + val_size))
test_dataset = dataset.select(range(train_size + val_size))

In [3]:
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import AutoModelForCausalLM

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token  # LLaMA는 기본적으로 pad_token이 없으므로 eos_token 사용

# 데이터 포맷 변환 함수
def format_llama_prompt(example):
    prompt = f"### 질문: {example['instruction']}\n### 답변: {example['output']}"
    return {"text": prompt}

# 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )

# Train, Validation, Test 데이터셋 변환
train_dataset = train_dataset.map(format_llama_prompt).map(tokenize_function, batched=True)
val_dataset = val_dataset.map(format_llama_prompt).map(tokenize_function, batched=True)
test_dataset = test_dataset.map(format_llama_prompt).map(tokenize_function, batched=True)

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/70 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import AutoModelForCausalLM

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./llama-3.2-1B-sft",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=30,
    eval_steps=30,
    logging_steps=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=True,  # Mixed Precision 사용 (GPU 환경)
    save_total_limit=2,
    push_to_hub=False,
    report_to="wandb", 
    run_name="llama-3.2-1B-finetuning",  
)

# SFTTrainer 생성
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

# 모델 학습 시작
trainer.train()


/home/paradeigma/workspace/python/sparta_coding/.env/lib/python3.9/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_24461/2366907953.py:26: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/home/paradeigma/workspace/python/sparta_coding/.env/lib/python3.9/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 

KeyboardInterrupt: 